# Model 1

## Pakete und Daten laden

In einem ersten Schritt laden wir die nöten Pakete und laden die Daten.

In [ ]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance
from pathlib import Path

target = Path("C:/Users/mhu/Documents/github/topic_model_it")
os.chdir(target)

data_model_1 = pd.read_csv("data/informatikkurse.csv")
print(data_model_1.head())

## Reproduzierbarkeit durch Seeds
Im folgenden fixieren wir die Zufallszahlen-Generatoren aller beteiligten Bibliotheken auf den Wert 11. Dies stellt sicher, dass die Experimente bei jedem Durchlauf identische Ergebnisse liefern.

In [ ]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

## Docs

Wir generieren als nächsten die Dokumente, die das Eingangsmaterial für das Topic-Model bilden.

In [18]:
# Anzeige der Spaltennamen von data_model_1
print(data_model_1.columns.tolist())

data_model_1 = data[["veranstaltung_titel", "kursbeschreibung"]].copy()
data_model_1.head()

data_model_1 = data_model_1.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
data_model_1['titel_kursbesch'] = data_model_1['veranstaltung_titel'] + ' ' + data_model_1['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = data_model_1['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

['veranstaltung_titel', 'kursbeschreibung', 'titel_kursbesch']


## Stopwords

In [ ]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

# Model

Wir definieren unser Model.

In [ ]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

CUDA available: True
Current device: cuda:0


## Model-Settings
Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

In [20]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=15,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

In [ ]:
# BERTopic initialisieren
topic_model = BERTopic(
  embedding_model=embedding_model,
  #min_topic_size=10,
  nr_topics=25, 
  language="multilingual",
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 15,
  representation_model=representation_model
)

Training

In [ ]:
topic_model_quanten = topic_model.fit(docs)

In [ ]:
# Korrekt & knapp
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

 
client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(client=client, model="gpt-4o-mini", delay_in_seconds=5)

# nutze dieselben Texte wie beim Fit (z. B. `docs`)
topic_model.update_topics(docs, representation_model=rep)
topic_model.get_topic_info()

 

In [ ]:
# # BERTopic auf Test-Daten anwenden
topics, probs = topic_model_quanten.transform(test_docs)

In [ ]:
topic_model_quanten.get_topic_info()

In [ ]:
# Outlier reduzieren
topics = topic_model_quanten.reduce_outliers(test_docs, topics)

In [ ]:
topics

In [ ]:
# Resultierende Topic-Nummern mit den Representations (= relevante Begriffe) zu einem Datensatz kombinieren
dataframe_with_results_left = pd.DataFrame(topics, columns = ["Topic"])
dataframe_with_results_right = pd.DataFrame(topic_model_quanten.get_topic_info().set_index('Topic')[['Representation']])
dataframe_with_results = dataframe_with_results_left.join(dataframe_with_results_right, on="Topic")

In [ ]:
row_number = 0
metric = 0
while row_number < len(ground_truth):
  # Den Goldstandard in eine Liste von Keywords umwandeln
  ground_truth_current_iteration = ground_truth[row_number].split(", ")
  result_current_iteration = dataframe_with_results.at[row_number, "Representation"]
  # Überprüfen, ob irgendein Begriff aus dem Resultat im Goldstandard zum Text vorkommt (1 = ja, 0 = nein)
  if any(element in result_current_iteration for element in ground_truth_current_iteration):
      metric += 1
  else: 
      metric += 0
      print(result_current_iteration)
      print(ground_truth_current_iteration)
      print("--------------------------------------------------------------------------------------------------")
  row_number = row_number+1

metric_score = metric/row_number
print(metric_score)